# Exercise 3: Data Lake on S3

In [1]:
from pyspark.sql import SparkSession
import os
import configparser

# Make sure that your AWS credentials are loaded as env vars

In [2]:
config = configparser.ConfigParser()

#Normally this file should be in ~/.aws/credentials
config.read_file(open('aws/credentials.cfg'))

os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

# Create spark session with hadoop-aws package

In [3]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

# Load data from S3

In [4]:
df = spark.read.csv("s3a://udacity-cbohara/pagila/payment/payment.csv")

In [6]:
df.printSchema()
df.limit(5)

root
 |-- _c0: string (nullable = true)



,_c0
0,payment_id;customer_id;staff_id;rental_id;amou...
1,16050;269;2;7;1.99;2017-01-24 21:40:19.996577+00
2,16051;269;1;98;0.99;2017-01-25 15:16:50.996577+00
3,16052;269;2;678;6.99;2017-01-28 21:44:14.99657...
4,16053;269;2;703;0.99;2017-01-29 00:58:02.99657...


# Infer schema, fix header and separator

In [7]:
df = spark.read.csv("s3a://udacity-cbohara/pagila/payment/payment.csv",sep=";", inferSchema=True, header=True)

In [8]:
df.printSchema()
df.show(5)

root
 |-- payment_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- rental_id: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- payment_date: string (nullable = true)

+----------+-----------+--------+---------+------+--------------------+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|
+----------+-----------+--------+---------+------+--------------------+
|     16050|        269|       2|        7|  1.99|2017-01-24 21:40:...|
|     16051|        269|       1|       98|  0.99|2017-01-25 15:16:...|
|     16052|        269|       2|      678|  6.99|2017-01-28 21:44:...|
|     16053|        269|       2|      703|  0.99|2017-01-29 00:58:...|
|     16054|        269|       1|      750|  4.99|2017-01-29 08:10:...|
+----------+-----------+--------+---------+------+--------------------+
only showing top 5 rows



# Fix the data yourself 

In [12]:
# common approach in Spark to load in all functions as F
import  pyspark.sql.functions as F
dfPayment = df.withColumn("payment_date", F.to_timestamp("payment_date"))
dfPayment.printSchema()
dfPayment.show(5)

root
 |-- payment_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- rental_id: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- payment_date: timestamp (nullable = true)

+----------+-----------+--------+---------+------+--------------------+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|
+----------+-----------+--------+---------+------+--------------------+
|     16050|        269|       2|        7|  1.99|2017-01-24 21:40:...|
|     16051|        269|       1|       98|  0.99|2017-01-25 15:16:...|
|     16052|        269|       2|      678|  6.99|2017-01-28 21:44:...|
|     16053|        269|       2|      703|  0.99|2017-01-29 00:58:...|
|     16054|        269|       1|      750|  4.99|2017-01-29 08:10:...|
+----------+-----------+--------+---------+------+--------------------+
only showing top 5 rows



# Extract the month

In [13]:
dfPayment = dfPayment.withColumn("month", F.month("payment_date"))
dfPayment.show(5)

+----------+-----------+--------+---------+------+--------------------+-----+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|month|
+----------+-----------+--------+---------+------+--------------------+-----+
|     16050|        269|       2|        7|  1.99|2017-01-24 21:40:...|    1|
|     16051|        269|       1|       98|  0.99|2017-01-25 15:16:...|    1|
|     16052|        269|       2|      678|  6.99|2017-01-28 21:44:...|    1|
|     16053|        269|       2|      703|  0.99|2017-01-29 00:58:...|    1|
|     16054|        269|       1|      750|  4.99|2017-01-29 08:10:...|    1|
+----------+-----------+--------+---------+------+--------------------+-----+
only showing top 5 rows



# Computer aggregate revenue per month

In [14]:
dfPayment.createOrReplaceTempView("payment")
spark.sql("""
    SELECT month, sum(amount) as revenue
    FROM payment
    GROUP by month
    order by revenue desc
""").show()

+-----+------------------+
|month|           revenue|
+-----+------------------+
|    4|28559.460000003943|
|    3|23886.560000002115|
|    2| 9631.879999999608|
|    1| 4824.429999999856|
|    5|  514.180000000001|
+-----+------------------+



# Fix the schema

In [34]:
from pyspark.sql.types import StructType, StructField, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
paymentSchema = StructType([
    StructField("payment_id",Int()),
    StructField("customer_id",Int()),
    StructField("staff_id",Int()),
    StructField("rental_id",Int()),
    StructField("amount",Dbl()),
    StructField("payment_date",Date()),
])

In [35]:
dfPaymentWithSchema = spark.read.csv("s3a://udacity-dend/pagila/payment/payment.csv",sep=";", schema=paymentSchema, header=True)


In [37]:
dfPaymentWithSchema.printSchema()
df.show(5)

root
 |-- payment_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- rental_id: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- payment_date: date (nullable = true)

+----------+-----------+--------+---------+------+--------------------+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|
+----------+-----------+--------+---------+------+--------------------+
|     16050|        269|       2|        7|  1.99|2017-01-24 21:40:...|
|     16051|        269|       1|       98|  0.99|2017-01-25 15:16:...|
|     16052|        269|       2|      678|  6.99|2017-01-28 21:44:...|
|     16053|        269|       2|      703|  0.99|2017-01-29 00:58:...|
|     16054|        269|       1|      750|  4.99|2017-01-29 08:10:...|
+----------+-----------+--------+---------+------+--------------------+
only showing top 5 rows



In [39]:
dfPaymentWithSchema.createOrReplaceTempView("payment")
spark.sql("""
    SELECT month(payment_date) as m, sum(amount) as revenue
    FROM payment
    GROUP by m
    order by revenue desc
""").show()

+---+------------------+
|  m|           revenue|
+---+------------------+
|  4|28559.460000003943|
|  3|23886.560000002115|
|  2| 9631.879999999608|
|  1| 4824.429999999856|
|  5|  514.180000000001|
+---+------------------+

